# i am to create a vector database that helps chumk a pdf file and retrive from the vector database 

## I will be using faiss  then chromma

## Using chroma

In [5]:
import chromadb
from chromadb.utils import embedding_functions
from chromadb.utils import embedding_functions

#use sentence-transformer embedding function 
sentence_trans_embeding_func= embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")


In [6]:
client= chromadb.PersistentClient("./chroma_db")

# collection= client.get_or_create_collection(name="python_for_dummies_collection",metadata={"category":"Programming"},embedding_function=sentence_trans_embeding_func)

In [3]:
collection= client.get_collection(name="python_for_dummies_collection")

In [10]:
collection.name

'python_for_dummies_collection'

In [7]:
def chunk_by_sentences(text, max_chunk_size=500):
    """
    Split text into chunks by sentences, keeping sentences intact.
    
    Args:
        text: The text to chunk
        max_chunk_size: Maximum characters per chunk
    
    Returns:
        List of text chunks
    """
    # Simple sentence splitting (split on . ! ?)
    import re
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        # Check if adding this sentence would exceed max size
        if len(current_chunk) + len(sentence) > max_chunk_size and current_chunk:
            # Save current chunk and start new one
            chunks.append(current_chunk.strip())
            current_chunk = sentence
        else:
            # Add sentence to current chunk
            current_chunk += " " + sentence if current_chunk else sentence
    
    # Don't forget the last chunk
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

In [12]:
#now i want to chunk the pdf file 
# to chunk i first need to know what the content of the pdf is , so i need something to read the pdf 
import PyPDF2
import os
file_path=r'C:\Users\User\Desktop\Working_with_LLMS\RAG\Python For Dummies.pdf'
file_name= os.path.basename(file_path)
chunks=[]
with open(file_path,'rb') as f:
    pdf_reader=PyPDF2.PdfReader(f)
    num_pages= len(pdf_reader.pages)

    for page_num in range(num_pages):
        page=pdf_reader.pages[page_num]
        text= page.extract_text()

        #chunk the page text
        page_chunks=chunk_by_sentences(text,max_chunk_size=300)
        for chunk in page_chunks:
            chunks.append(chunk)

In [13]:
#now that the pdf is chunked i want to save it inside my chromma collection

In [14]:
ids=[f"doc_{i}" for i in range(len(chunks))]
collection.add(ids=ids,documents=chunks)

C:\Users\User\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [02:19<00:00, 596kiB/s] 


In [8]:
results=collection.query(query_texts=["What are integers"],n_results=4,include=["embeddings", "documents", "metadatas", "distances"])

In [17]:
print("Query: What is artificial intelligence?\n")
print("Top 3 results:\n")

retrieved_text=""
for i, (doc, metadata, distance) in enumerate(zip(
    results['documents'][0],
    results['metadatas'][0],
    results['distances'][0]
), 1):
    
    retrieved_text += doc
    # print(f"{i}. (Distance: {distance:.4f})")
    # print(f"   Document: {doc}")
    # print(f"   Metadata: {metadata}")
    # print()
print(retrieved_text)

Query: What is artificial intelligence?

Top 3 results:

After all, humans can use just one kind 
of number. To understand the need for multiple 
number types, you have to understand a little 
about how a computer works with numbers. An integer is stored in the computer as simply a 
series of bits that the computer reads directly.For example, the value 1 is a whole number, 
so it’s an integer. On the other hand, 1.0 isn’t a whole number; it has a deci -
mal part to it, so it’s not an integer. Integers are represented by the int 
data type. As with storage boxes, variables have capacity limits.For now, all you really need to know is that integers support different 
numeric bases.86 Part II: Talking the Talk 
because each form requires a different kind of processing. The following sec -
tions describe the integer, floating-point, and complex number classes of data 
types that Python supports. Integers
Any whole number is an integer.


In [18]:
import os 
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("openai_key")

if not openai_api_key:
    print("Warning: OpenAikey not found.Set it in env file")

else:
    print("API key loaded successfully")

API key loaded successfully


In [21]:
import numpy as np
from openai import OpenAI
#4. Generate response 
client=OpenAI(api_key=openai_api_key)
question= "what are integers"
prompt= f"""
You are a helpful AI assistant. Answer the user's question based on the context provided below.

Important guidelines:
- Use ONLY information from the context
- If the answer is not in the context, say "I don't have enough information to answer this question."
- Be concise and accurate

Context:
{retrieved_text}

Question: {question}

Answer:
"""


response= client.chat.completions.create(model="gpt-3.5-turbo",messages=[{"role":"user","content":prompt}],temperature=0)
print(response)

ChatCompletion(id='chatcmpl-CmI6T1e5YRHI58OKiJsJyPc8TTIo2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Integers are whole numbers that are stored in a computer as a series of bits. They are represented by the int data type and support different numeric bases.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1765625369, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=31, prompt_tokens=277, total_tokens=308, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
